In [1]:
import pandas as pd
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import ShuffleSplit, cross_validate
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn import metrics

from sklearn import svm

nltk.download("stopwords")


[nltk_data] Downloading package stopwords to /home/marvin-
[nltk_data]     linux/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Carregar o dataframe

In [2]:
df = pd.read_csv(
    "../data/raw/NoThemeTweets.csv", usecols=["tweet_text", "sentiment"]
).sample(30000)


df.shape


(30000, 2)

In [3]:
corpus_train = df.tweet_text[2000:].to_list()
labels_train = df.sentiment[2000:].replace({"Positivo": 1, "Negativo": 0}).to_list()


corpus_test = df.tweet_text[:2000].to_list()
labels_test = df.sentiment[:2000].replace({"Positivo": 1, "Negativo": 0}).to_list()

stop_words = nltk.corpus.stopwords.words("portuguese") + ["https"] + ["co"]



In [4]:
from nltk.tokenize import TweetTokenizer

tweet_tokenizer = TweetTokenizer()

pipeline = Pipeline(
    steps=[
        (
            "vectorizer",
            CountVectorizer(analyzer="word", stop_words=stop_words, max_features=1000),
        ),
        ("pca", TruncatedSVD(500)),
        ("clf", KNeighborsClassifier(n_neighbors=11)),
    ]
)

pipeline2 = Pipeline(
    steps=[
        (
            "vectorizer",
            CountVectorizer(
                analyzer="word",
                stop_words=stop_words,
                max_features=1000,
                tokenizer=tweet_tokenizer.tokenize,
            ),
        ),
        ("pca", TruncatedSVD(500)),
        ("clf", KNeighborsClassifier(n_neighbors=11)),
    ]
)


pipeline3 = Pipeline(
    steps=[
        (
            "vectorizer",
            CountVectorizer(
                analyzer="word",
                stop_words=stop_words,
                max_features=1000,
                tokenizer=tweet_tokenizer.tokenize,
            ),
        ),
        ("clf", MultinomialNB()),
    ]
)




pipeline4 = Pipeline(
    steps=[
        (
            "vectorizer",
            CountVectorizer(
                analyzer="word",
                stop_words=stop_words,
                max_features=1000,
                tokenizer=tweet_tokenizer.tokenize,
            ),
        ),
        ("clf", svm.SVC(kernel='linear')),
    ]
)


In [5]:
pipeline.fit(corpus_train, labels_train)
pipeline2.fit(corpus_train, labels_train)
pipeline3.fit(corpus_train, labels_train)
pipeline4.fit(corpus_train, labels_train)


Pipeline(steps=[('vectorizer',
                 CountVectorizer(max_features=1000,
                                 stop_words=['de', 'a', 'o', 'que', 'e', 'é',
                                             'do', 'da', 'em', 'um', 'para',
                                             'com', 'não', 'uma', 'os', 'no',
                                             'se', 'na', 'por', 'mais', 'as',
                                             'dos', 'como', 'mas', 'ao', 'ele',
                                             'das', 'à', 'seu', 'sua', ...],
                                 tokenizer=<bound method TweetTokenizer.tokenize of <nltk.tokenize.casual.TweetTokenizer object at 0x7f4328d1f5b0>>)),
                ('clf', SVC(kernel='linear'))])

In [6]:
# pipeline3.fit(corpus_train, labels_train)
# y_hat3 = pipeline3.predict(corpus_test)
# y_hat3

In [7]:
y_hat = pipeline.predict(corpus_test)
y_hat2 = pipeline2.predict(corpus_test)
y_hat3 = pipeline3.predict(corpus_test)
y_hat4 = pipeline4.predict(corpus_test)



In [8]:
accuracy_score(labels_test, y_hat)


0.6635

In [9]:
accuracy_score(labels_test, y_hat2)


0.9935

In [10]:
accuracy_score(labels_test, y_hat3)

0.997

In [11]:
accuracy_score(labels_test, y_hat4)

0.9995

In [12]:
from nltk.tokenize import TweetTokenizer

tweet_tokenizer = TweetTokenizer()

cv = CountVectorizer(analyzer="word", stop_words=stop_words, max_features=1500, tokenizer=tweet_tokenizer.tokenize)

freq_train = cv.fit_transform(corpus_train)


freq_train

<28000x1500 sparse matrix of type '<class 'numpy.int64'>'
	with 184854 stored elements in Compressed Sparse Row format>

In [13]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()

model.fit(freq_train, labels_train)

model


freq_train


<28000x1500 sparse matrix of type '<class 'numpy.int64'>'
	with 184854 stored elements in Compressed Sparse Row format>

In [14]:
teste = [
         "Não é um bom dia para começar a programar",
         "Estou ficando muito cansado",
         "Não consigo fazer nada",
         "Que dia lindo, hoje é dia de programar",
         "É coisa, é coisa pura, é coisa maravilhosa",
         "Bolsonaro é um cara horrível",
         "Estou querendo morrer",
         "Abençoa senhor a minha vida bolsonaro",
        ]








# accuracy_score(labels, y_hat2)

# for t, c in zip (teste,model.predict(teste)):
#     print(t)



# y_hat_tes

In [15]:
resultados = cross_val_predict(pipeline4, corpus_train, labels_train, cv=10)




In [16]:
resultados2 = cross_val_score(pipeline4, corpus_train, labels_train, cv=10)

In [17]:
metrics.accuracy_score(labels_train, resultados)

0.9983214285714286

In [22]:
resultados3 = cross_validate(pipeline4, corpus_train, labels_train, cv=10)

In [ ]:
resultados3